In [ ]:
import pathlib
import os
if 'TSL_SCHOOL_DIR' in os.environ:
     if any( (p/".git").is_dir() for p in
(pathlib.Path(".").absolute().resolve()/"dummy").parents ):
         raise RuntimeError('Please copy notebook to a work directory')

# Structural Optimization  



The equilibrium positions and cell size in DFT might be slightly different than the experimental ones. Those coordinates that represent the true minimum for the DFT calculation are usually referred to as theoretical lattice parameters. It is particularly important that the system sits in its theoretical minimum if you want to compute the phonons afterwards because the phonons are computed as perturbations around an equilibrium minimum.   

In Quantum ESPRESSO it is possible to automatically optimize the internal atomic positions and cell vectors by changing the calculation type. There are two possible options 'relax' it will keep the cell fix and optimize the internal coordinates and 'vc-relax' that will optimize both the internal coordinates and the cell. 

These two calculations require additional NAMELISTS namely: &IONS and &CELLS, in which you can specify details on the algorithm used for the minimization. 


      Let's see how this slightly more complicated input file looks like:

In [1]:
cat pw.vcrelax.silicon.in

&CONTROL
  ! vc-relax stands for variable-cell relaxation, it will change the lattice parameters and internal position to find the minimum
  calculation = 'vc-relax'
  prefix = 'si'
  outdir = './'
  pseudo_dir = './pseudo/'
  verbosity = 'high'  
  ! you may want to specify the convergence threshold on the forces 
  forc_conv_thr=1.0d-4
  ! and you may want to specify the convergence threshold on the total energy
  etot_conv_thr=1.0d-6
/
&SYSTEM
  ecutwfc =   30
  ecutrho =   240
  ibrav = 0
  nat = 2
  ntyp = 1
/
&ELECTRONS
  conv_thr =   4.0000000000d-10
  electron_maxstep = 100
  mixing_beta =   0.4
/
&IONS
! here you can specify details on the minimuzation algorithm for the internal positions
/
&CELL
! here you can specify details on the minimuzation algorithm for the lattice parameters
/
ATOMIC_SPECIES
Si     28.0855 Si.pbe-n-rrkjus_psl.1.0.0.UPF
K_POINTS automatic
8 8 8 0 0 0
ATOMIC_POSITIONS angstrom
Si      0.0000000   0.0000000   0.0000000
Si      1.3452500   1.3452500   1.34

Now you can run QUANTUM-Espresso from the terminal 

     pw.x < pw.vcrelax.silicon.in > pw.vcrelax.silicon.out
For parallel execution

     mpirun -np 4 pw.x -inp pw.vcrelax.silicon.in > pw.vcrelax.silicon.out

The code will converge the SCF cycle, compute forces and stresses and change the coordinates and cell parameters accordingly. We can see the converged energy for each ionic step by doing

In [4]:
! grep ! pw.vcrelax.silicon.out

!    total energy              =     -22.83762230 Ry
!    total energy              =     -22.83917601 Ry
!    total energy              =     -22.83967228 Ry
!    total energy              =     -22.83967501 Ry
!    total energy              =     -22.83967501 Ry
!    total energy              =     -22.83971503 Ry


as you can see the code performed 6 steps of geometry optimization converging the SCF cycle for each step

Now we can compare the original and final coordinates, the original coordinates are:

In [5]:
! grep -A6 "ATOMIC_POSITIONS" pw.vcrelax.silicon.in

ATOMIC_POSITIONS angstrom
Si      0.0000000   0.0000000   0.0000000
Si      1.3452500   1.3452500   1.3452500
CELL_PARAMETERS angstrom
  2.6905000   2.6905000   0.0000000 
  2.6905000   0.0000000   2.6905000 
  0.0000000   2.6905000   2.6905000 


The optimized atomic positions are:

In [6]:
! grep -A2 "ATOMIC_POSITIONS" pw.vcrelax.silicon.out | tail -3

ATOMIC_POSITIONS (angstrom)
Si            0.0000000000       -0.0000000000        0.0000000000
Si            1.3672162898        1.3672162898        1.3672162898


while the optimized cell is:

In [7]:
! grep -A3 "CELL_PARAMETERS" pw.vcrelax.silicon.out | tail -4

CELL_PARAMETERS (angstrom)
   2.734432580   2.734432580   0.000000000
   2.734432580   0.000000000   2.734432580
   0.000000000   2.734432580   2.734432580


The cell is slightly larger than the experimental one and the atomic positions changed to reflect this expansion but they relative position remained the same because it is fixed by symmetry. In a general case where there are no symmetries constraining the position they will also change relative to each other.

We can have a look at the evolution of the structure using Xcrysden 

     xcrysden --pwo pw.vcrelax.silicon.out